In [1]:
# Global reordering analyses



In [1]:
using Pkg
using Eirene
using MAT
using CSV
using GraphPlot
using LightGraphs
using DataFrames
using JLD
using Plots
using Measures
using StatsPlots
using RCall
using PyCall
using Statistics
using StatsBase
using HypothesisTests
using LinearAlgebra
using MultivariateStats

# Call R functions
R"library(TDA)"
R"source('~/Dropbox/Top Sim and Homog/Scripts/bottleneck_computations_functions2.R')"
R"library(igraph)"
R"library(ggplot2)"
R"source('~/Dropbox/Top Sim and Homog/Scripts/local_network_functions.R')"




HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-9215602799624512293\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-7211012718290195272\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio/main\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n        console.warn(\"Loading WebIO Jupyter extension on an ad-hoc basis. Consider enabling the WebIO nbextension for a stabler experience (this should happen automatically when building WebIO).\");\n        require([\"/assetserver/b9b099db28a7ba8f329539eab76be3cb20183848-dist/main.js\"], function (webIOModule) {\n            webIOModule.load_ipython_extension();\n        });\n        warning_div.innerHTML = \"<strong>Loading WebIO Jupyter extension on an ad-hoc basis. Consider enabling the WebIO nbextension for a stabler experience (this should happen automatically when building WebIO).</strong>\";\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<div\n    id=\"webio-warning-9215602799624512293\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    <!-- TODO: link to installation docs. -->\n</div>\n")

┌ Warning: RCall.jl: 
│ Attaching package: ‘igraph’
│ 
│ The following objects are masked from ‘package:stats’:
│ 
│     decompose, spectrum
│ 
│ The following object is masked from ‘package:base’:
│ 
│     union
│ 
└ @ RCall /Users/annsizemore/.julia/packages/RCall/ffM0W/src/io.jl:113


RObject{VecSxp}
$value
function (Xdata, Ydata, filename) 
{
    df <- data.frame(xvar = Xdata, yvar = Ydata)
    plt1 <- ggplot(df, aes(x = xvar, y = yvar)) + geom_smooth(method = lm) + 
        theme_bw()
    ggplot2::ggsave(paste(filename, "1.pdf", sep = ""), device = "pdf", 
        width = 4, height = 4)
    print("saved 1")
    plt2 <- ggplot(df, aes(x = xvar, y = yvar)) + stat_density2d(geom = "tile", 
        aes(fill = ..density..^0.25, alpha = 1), contour = FALSE) + 
        scale_fill_gradientn(colours = colorRampPalette(c("white", 
            blues9))(256)) + coord_fixed() + geom_smooth(method = lm) + 
        theme_bw()
    pdf(paste(filename, "2.pdf", sep = ""))
    print(plt2)
    dev.off()
    print("saved 2")
}

$visible
[1] FALSE



In [2]:
## Functions

function toWeightedAdj_byweight(network_und,node_weights)
    #Assume that HIGHER node weight means born earlier-- this means we
    # want to keep the MINIMUM
    nNodes = length(node_weights)
    t0 = network_und.*node_weights
    t1 = deepcopy(t0)
    for i in collect(1:nNodes)
        for j in collect(i:nNodes)
            t1[i,j] = minimum([t0[i,j],t0[j,i]])
            t1[j,i] = t1[i,j]
        end
    end
    t1
end

# To calculate the bettiCurves from the barcodes
function bettiCurveFromBarcode(barcode_array,nNodes,nmats,maxDim)
    
    nNodes = Int(nNodes)
    nmats = Int(nmats)
    maxDim = Int(maxDim)
    bettiBar = zeros(nmats,maxDim)
    bettiCurve = zeros(nmats,nNodes+1,maxDim)
    birthCurve = zeros(nmats,nNodes,maxDim)
    deathCurve = zeros(nmats,nNodes,maxDim)
    

    for dimn in collect(1:maxDim)
        dimn = Int(dimn)
       
        for matn in collect(1:nmats)
            matn = Int(matn)
            bb = 0
            currentCurve = barcode_array[matn,:]
            currentCurveDim = currentCurve[dimn]
            for barn in collect(1:size(currentCurveDim,1))
               

                # Add to birth curve
                birthCurve[matn,Int(currentCurveDim[barn,1]),dimn] = birthCurve[matn,Int(currentCurveDim[barn,1]),dimn] .+1


                if currentCurveDim[barn,2]>nNodes
                  
                    bettiCurve[matn,Int(currentCurveDim[barn,1]):Int(nNodes+1),dimn] = bettiCurve[matn,Int(currentCurveDim[barn,1]):Int(nNodes+1),dimn] .+1
                    bb = bb+(nNodes+1-currentCurveDim[barn,1])
                    else 
                   
                    bettiCurve[matn,Int(currentCurveDim[barn,1]):Int(currentCurveDim[barn,2]),dimn] = bettiCurve[matn,Int(currentCurveDim[barn,1]):Int(currentCurveDim[barn,2]),dimn].+1
                    deathCurve[matn,Int(currentCurveDim[barn,2]),dimn] = deathCurve[matn,Int(currentCurveDim[barn,2]),dimn] .+1
                    bb = bb+(currentCurveDim[barn,2] - currentCurveDim[barn,1])

                end
            end
            
            bettiBar[matn,dimn] = deepcopy(bb)
        end
    end
    
    return bettiCurve, birthCurve, deathCurve, bettiBar 
end


## Calculating edge density
function calculateEdgeQuantity(jadj_array)
    nmats = size(jadj_array,3)
    nNodes = size(jadj_array,1)
    edgeQuantity = zeros(nmats,nNodes)
    
    # Loop through to compute number of edges at each filtration step
    for matn in collect(1:nmats)
        adj = jadj_array[:,:,matn];
        for noden in collect(1:nNodes)
            edgesAdded = (LinearIndices(adj))[findall(adj.<= noden)]
            edgeQuantity[Int(matn),Int(noden)] = length(edgesAdded)/2
        end
    end
    edgeQuantity = edgeQuantity.-35   # get rid of self-loops = 0
    
end

function calculateDegreesFiltration(jadj_array,nNodes)
    nmats = size(jadj_array,3)
    nNodes = size(jadj_array,1)
    degree_array = zeros(nNodes,nNodes,nmats)
    
    jadj_array[jadj_array.==(nNodes*2)] .= 0
    
    
    # Loop through to compute number of edges at each filtration step
    for matn in collect(1:nmats)
        adj = jadj_array[:,:,matn];
        for noden in collect(1:nNodes)
            adj_i = deepcopy(adj)
            adj_i[adj_i.>noden] .= 0
            adj_i[adj_i.>0] .= 1
            degree_array[:,Int(noden),Int(matn)] = sum(adj_i,dims = 1)
        end
    end
    
    return degree_array
    
end


## Making weights from order
function orderToWeights(s_0_array,nNodes)
        s_wei_array = zeros(size(s_0_array))
    for i0 in collect(1:size(s_0_array,1))

        for n0 in collect(1:nNodes)
            n0 = Int(n0)
            s_wei_array[i0,n0] = indexin([n0],s_0_array[i0,:])[1]
        end
    end
    s_wei_array
end

function calculateTheoreticaMaxDistance(s_wei_array,n) 
    theo_dist= Vector{Float64}(undef,n)   #10100 or 24160
    for i1 in collect(1:size(s_wei_array,1))
        norm1 = norm(s_wei_array[1,:].-s_wei_array[i1,:],Inf)
        theo_dist[i1] = norm1
    end
    return theo_dist
end

## Making Diagrams from barcode for TDA in R
function makeDiagramFromBarcode(barcode_array,graphn,maxDim)

    diag = []
    nNodes = 70


    for d in collect(1:maxDim)
        
        bd = barcode_array[graphn,d]
        infs = findall(bd[:,2] .> nNodes)
        bd[infs,2] .= nNodes+1
        nbars = size(bd,1)
        
        if nbars>0
            bda = hcat(d*ones(nbars),bd)
        else
            bda = [d 0 0]
        end
                
                        
       
        
        if isequal(diag,[])
            diag = bda
            else
            diag = vcat(diag,bda)
        end
                
    end
    diag
end




function computeBNDistances_glob(barcode_array)

    nReps = 101
    nGraphs = 100
    maxDim = 3

    graphOriginals = collect(1:101:10100)
    distanceBN_array = zeros(10100,3)
    

    for nG in graphOriginals
               
        diagO = makeDiagramFromBarcode(barcode_array,nG,maxDim)
        
        for nR in collect(1:(nReps-1))
            diagR = makeDiagramFromBarcode(barcode_array,(nG+nR),maxDim)

            for nD in collect(1:3)
                distanceBN_dimn = R"computeBNDistance($diagO,$diagR,$nD)"
                distanceBN_array[(nG+nR),nD] = distanceBN_dimn
            end
        end
    end
    
    return distanceBN_array
end


            
function computeBNDistances_globSampled(barcode_array)

    nReps = 101
    nGraphs = 100
    maxDim = 3
    runs = collect(1:100)

    graphOriginals = collect(1:101:10100)
    distanceBN_array = zeros(10000,3)
    

    for nG in collect(1:nGraphs)
                    
        a = (nG-1)*nReps + 1
                    
        for runi in runs
                        
        p = sample(collect(a:(a+100)),2,replace = false)
        diaga = makeDiagramFromBarcode(barcode_array,p[1],maxDim)
        diagb = makeDiagramFromBarcode(barcode_array,p[2],maxDim)

            for nD in collect(1:3)
                distanceBN_dimn = R"computeBNDistance($diaga,$diagb,$nD)"
                distanceBN_array[((nG-1)*(nReps-1) + runi),nD] = distanceBN_dimn
            end
        end

    end
    
    return distanceBN_array
end

            
            
            
function computeBNDistances_Sampled(barcode_array,nGraphs)

    
    maxDim = 3
    distanceBN_array = zeros(10000,3)
    
    runs = collect(1:size(distanceBN_array)[1])

    for runi in runs
        p = sample(collect(1:nGraphs),2,replace = false)
        diaga = makeDiagramFromBarcode(barcode_array,p[1],maxDim)
        diagb = makeDiagramFromBarcode(barcode_array,p[2],maxDim)

        for nD in collect(1:3)
            distanceBN_dimn = R"computeBNDistance($diaga,$diagb,$nD)"
            distanceBN_array[runi,nD] = distanceBN_dimn
        end

    end
    
    return distanceBN_array
end
            

function computeBNDistances_local(barcode_array)

    nReps = 2416
    nGraphs = 10
    maxDim = 3
    nNodes = 70

    graphOriginals = collect(1:2416:24160)
    distanceBN_array = zeros(24160,3)

    for nG in graphOriginals
        diagO = makeDiagramFromBarcode(barcode_array,nG,maxDim)
        println(nG)
        for nR in collect(1:(nReps-1))
            diagR = makeDiagramFromBarcode(barcode_array,(nG+nR),maxDim)

            for nD in collect(1:3)
                distanceBN_dimn = R"computeBNDistance($diagO,$diagR,$nD)"
                distanceBN_array[(nG+nR),nD] = distanceBN_dimn
            end
        end
    end
    
    return distanceBN_array
end
    
    
function plotBarcode(allPIs,nNodes,graphN,maxDim,fontSize)

    nNodes = Int(nNodes)
    graphn = Int(graphN)
    maxDim = Int(maxDim)
    counter1 = 0
    pbar = plot(1:6,zeros(6),c=:black)
    
    colors = [:blue :green :red]
    for dim in collect(1:maxDim)

        barn = barcode_array[graphN, dim]
        barn = barn[sortperm(barn[:,1]),:]

        nbars = size(barn)[1]


        for cntr1 in collect(1:nbars)
            birth = barn[cntr1,1]
            death = barn[cntr1,2]

            plot!([birth, death],[cntr1+counter1, cntr1+counter1],c=colors[dim], legend = false,
                            xlim = (0,nNodes), ytickfont = font(fontSize), xtickfont = font(fontSize))
        end

        display(pbar)



        counter1 = counter1+nbars
    end

    return pbar
end

            
function strictLTvector(x)
    xVec = [x[i,j] for j in 1:size(x)[2]-1 for i in j+1:size(x)[1]]
    return xVec
end
            
function strictLTvector3D(x)
    xVec = [x[i,j,d] for d in 1:size(x)[3] for j in 1:size(x)[2]-1 for i in j+1:size(x)[1]]
    return xVec
end

strictLTvector3D (generic function with 1 method)

In [3]:
## Global reordering analysis

####### ---- stuff to define ------ #####
maxDim = 3
yLim = 60       # 70 for propprob, 6 for RG and abssin, 150 for ER, 80 for pa, 60 spgr
graph_name = "NF_spatialGrowth_b1a4_1218"

colors_orig = [RGB(0.1,0.1,1) RGB(0.1,1,0.1) RGB(1,0.1,0.1)]
colors_1 = [RGB(0.15,0.15,1) RGB(0.15,1,0.15) RGB(1,0.15,0.15)]
colors_2 = [RGB(0.19,0.19,1) RGB(0.19,1,0.19) RGB(1,0.19,0.19)]
###### ---------------------------- ######

In [4]:
## Load
graph_name_0 = "$(graph_name)"
graph_name_glob = "$(graph_name)_glob"
graph_name_local = "$(graph_name)_local"


## Load the graph and 1000 reps
filename_0 = "$graph_name_0"
z = load("Results/$(filename_0)_pis.jld")
barcode_array = z["barcode_array"]
dict1 = matread("Results/$(filename_0).mat")
s_0_array = dict1["s_0_array"]
nReps = dict1["nReps"]
nGraphs = dict1["nGraphs"]
nNodes = dict1["nNodes"]
jadj_array = dict1["jadj_array"]
nmats = size(s_0_array,1)

## Clear
dict0 = nothing
dict1 = nothing
z = nothing
GC.gc()

In [5]:
## Calculate Betti curve
bettiCurve, birthCurve, deathCurve = bettiCurveFromBarcode(barcode_array,nNodes,nmats,maxDim)

ribbon_std = dropdims(std(bettiCurve,dims = 1),dims = 1)
bettiCurve_mean = mean(bettiCurve,dims = 1)
bettiCurve_mean = dropdims(bettiCurve_mean,dims = 1)
birthCurveMean = dropdims(mean(birthCurve,dims = 1),dims = 1)
birthCurveStd = dropdims(std(birthCurve,dims = 1), dims = 1)
deathCurveMean = dropdims(mean(deathCurve, dims = 1), dims = 1)
deathCurveStd = dropdims(std(deathCurve,dims = 1), dims = 1)


## Pick one example growing graph to plot
adj = jadj_array[:,:,1]
adj[adj.==(nNodes*2)] .= 0
adj[adj.>0] .= 1 

## For persistence diagrams
barcode1 = barcode_array[1,1]
barcode1[barcode1.==(nNodes*2)] .= nNodes
barcode2 = barcode_array[1,2]
barcode2[barcode2.==(nNodes*2)] .= nNodes

## Calculate edge density along the filtration 
edgeQuantityArray = calculateEdgeQuantity(jadj_array)
nEdgesFinal = binomial(Int(nNodes),2)
edgeDensityArray = edgeQuantityArray./nEdgesFinal
edgeDensityMean = dropdims(mean(edgeDensityArray,dims=1),dims = 1)
edgeDensityStd = dropdims(std(edgeDensityArray,dims = 1),dims = 1)

## Calculate average degree along filtration 
degree_array = calculateDegreesFiltration(jadj_array,nNodes)
degree_array_mean = dropdims(mean(degree_array,dims = 3),dims = 3)




70×70 Array{Float64,2}:
 0.0  1.0  1.619  2.14   2.565  2.963  …  18.971  19.202  19.477  19.74 
 0.0  1.0  1.528  1.947  2.295  2.583     15.993  16.235  16.462  16.694
 0.0  0.0  1.147  1.519  1.864  2.153     15.192  15.408  15.613  15.826
 0.0  0.0  0.0    1.312  1.646  1.965     14.83   15.035  15.23   15.451
 0.0  0.0  0.0    0.0    1.452  1.776     14.576  14.779  14.994  15.189
 0.0  0.0  0.0    0.0    0.0    1.618  …  14.345  14.542  14.737  14.969
 0.0  0.0  0.0    0.0    0.0    0.0       14.187  14.399  14.593  14.774
 0.0  0.0  0.0    0.0    0.0    0.0       14.066  14.263  14.478  14.673
 0.0  0.0  0.0    0.0    0.0    0.0       13.857  14.066  14.262  14.439
 0.0  0.0  0.0    0.0    0.0    0.0       13.642  13.835  14.03   14.222
 0.0  0.0  0.0    0.0    0.0    0.0    …  13.636  13.835  14.043  14.228
 0.0  0.0  0.0    0.0    0.0    0.0       13.623  13.814  14.01   14.206
 0.0  0.0  0.0    0.0    0.0    0.0       13.216  13.393  13.603  13.808
 ⋮                         

In [6]:
## Repeats above then...
## Choose random sample of 10,000 pairs from the above run to compute distance between Betti curves

nGraphs = 1000
nSamples = 10000
bettiCurveDistances = zeros(nSamples,3)
maxDim = 3
for i0 in collect(1:nSamples)
    
    # Pick two Betti Curves
    p = sample(collect(1:nGraphs),2,replace = false)

    bcurve1 = bettiCurve[p[1],:,:]
    bcurve2 = bettiCurve[p[2],:,:]
    
    for di in collect(1:maxDim)
        
        bcurvediff = abs.(bcurve1[:,di].- bcurve2[:,di])
        # Calculate left Riemann integral (delta x = 1)
        # Could also just take the max difference between betti curves. Both reasonable.
        RInt = sum(bcurvediff)
        maxDiff = maximum(bcurvediff)
   
        bettiCurveDistances[i0,di] = maxDiff
        
    end
end

    
# Now bettiCurveDistances has our distances between randomly sampled original graphs.    

print("done")


done

In [ ]:
###### Load or compute bottleneck distances


#dict3 = matread("Results/$(graph_name_glob)_bottleneckDistances.mat")
#dmat_glob = dict3["distanceMat"]
#dict9 = matread("Results/$(graph_name)_bottleneckDistances.mat")
#dmat_sampled = dict9["distanceMat"]
#dict10 = matread("Results/$(graph_name_glob)_sampledDistances.mat")
#dmat_glob_sampledR = dict10["distanceMat"]

z = load("Results/$(graph_name_glob)_pis.jld")
barcode_array_glob = z["barcode_array"]

println("Computing BN Distances between orig and reordered")
dmat_glob = computeBNDistances_glob(barcode_array_glob)
println("Computing BN Distances between sampled orderings")
dmat_glob_sampledR = computeBNDistances_globSampled(barcode_array_glob)
println("Computing BN Distances between sampled originals")
dmat_sampled = computeBNDistances_Sampled(barcode_array,size(barcode_array)[1])




######## ----- ######
makeBox = 2
colors_orig = [RGB(0.2,0.3,0.6) RGB(0.23,0.6,0.3) RGB(0.6,0.2,0.4)]
colors_r = [RGB(0.3,0.5,1) RGB(0.23,0.9,0.4) RGB(0.9,0.3,0.5)]

######## ----- ######

dict2 = matread("Results/$(graph_name_glob).mat")
s_0_array_glob = dict2["s_0_array"]
nNodes = dict2["nNodes"]
nGraphs = dict2["nGraphs"]
nReps = dict2["nReps"]

dict2 = nothing
dict3 = nothing
dict4 = nothing
GC.gc()
print("loaded relevant data")
println(nGraphs)
println(nReps)




println("Done loading more")

Computing BN Distances between orig and reordered


In [8]:
#### Actually compute things I'll need


s_wei_array_glob = orderToWeights(s_0_array_glob,nNodes)
theo_dist_glob = calculateTheoreticaMaxDistance(s_wei_array_glob,10100) 


inds1 = collect(0:nReps:(nReps*nGraphs-1))
inds1 = inds1 .+1
inds1 = Int.(inds1)
barcode_array_glob_orig = barcode_array_glob[inds1,:]
indsr = setdiff(1:(nReps*nGraphs),inds1)
indsr = Int.(indsr)
barcode_array_glob_r = barcode_array_glob[indsr,:]

## Make betti curve and such
println("Constructing Betti curves...")
bettiCurve_orig, birthCurve_orig, deathCurve_orig, bettiBar_orig = bettiCurveFromBarcode(barcode_array_glob_orig,nNodes,nGraphs,maxDim)
bettiCurve_r, birthCurve_r, deathCurve_r, bettiBar_r = bettiCurveFromBarcode(barcode_array_glob_r,nNodes,10000,maxDim)
bettiCurve_glob_all, birthCurve_glob_all, deathCurve_glob_all, bettiBar_glob_all = bettiCurveFromBarcode(barcode_array_glob,nNodes,10100,mac)
println("Done constructing Betti curves")
bettiCurve_orig_mean = dropdims(mean(bettiCurve_orig,dims = 1),dims = 1)
bettiCurve_r_mean = dropdims(mean(bettiCurve_r,dims = 1), dims = 1)
bettiCurveStd_orig = dropdims(std(bettiCurve_orig,dims = 1),dims = 1)
bettiCurveStd_r = dropdims(std(bettiCurve_r,dims = 1),dims = 1)


## Calculate differences between betti curves
bettiCurveDistances_glob = zeros(10000,3)
maxDim = 3
nGraphs = Int(nGraphs)
for i0 in collect(1:nGraphs)
    
    bcurve1 = bettiCurve_orig[i0,:,:]

    for r0 in collect(1:(nReps-1))
        
        repG = (i0-1)*nGraphs + r0
        bcurve2 = bettiCurve_r[Int(repG),:,:]

        for di in collect(1:maxDim)

            bcurvediff = abs.(bcurve1[:,Int(di)].- bcurve2[:,di])
            
            # Calculate left Riemann integral (delta x = 1)
            # Could also just take the max difference between betti curves. Both reasonable.
            #RInt = sum(bcurvediff)
            maxDiff = maximum(bcurvediff)

            bettiCurveDistances_glob[Int(repG),di] = maxDiff
        end
    end
end

    
## Calculate differences between betti curves keeping topology same
bettiCurveDistances_globSampled = zeros(10000,3)
nSamples = 100

for i0 in collect(1:nGraphs)
    for r0 in collect(1:nSamples)
        gs = sample(collect(1:100),2, replace = false)
        g1 = gs[1] + (i0-1)*100
        println(g1)
        g2 = gs[2]
        println(g2)
        bcurve1 = bettiCurve_r[Int(g1),:,:]
        bcurve2 = bettiCurve_r[Int(g2),:,:]
        
        for di in collect(1:maxDim)

            bcurvediff = abs.(bcurve1[:,Int(di)].- bcurve2[:,di])
            
            # Calculate left Riemann integral (delta x = 1)
            # Could also just take the max difference between betti curves. Both reasonable.
            RInt = sum(bcurvediff)
            maxDiff = maximum(bcurvediff)
            t1 = (i0-1)*100 + r0

            bettiCurveDistances_globSampled[Int(t1),di] = maxDiff
        end
    end
end




## Need to create boxplot for distances
dmat_glob_r = dmat_glob[indsr,:]

## Create boxplot for 1-distance/theo distance


## Need to create boxplot for difference in avg birth times

## Need to create boxplot for difference in avg death times

simMat_glob = 1 .- dmat_glob./theo_dist_glob
simMat_glob_r = simMat_glob[indsr,:]

println("The maximum Betti curve distance between sampled graphs is $(maximum(bettiCurveDistances))")
println("The max Betti curve distance between orig and reordered is $(maximum(bettiCurveDistances_glob))")
println("The max Betti curve distance between reorderings is $(maximum(bettiCurveDistances_globSampled))")
println("Max barcode distance between reorderings is $(maximum(dmat_glob_sampledR))")

Constructing Betti curves...
Done constructing Betti curves
The maximum Betti curve distance between sampled graphs is 83.0
The max Betti curve distance between orig and reordered is 42.0
The max Betti curve distance between reorderings is 69.0
Max barcode distance between reorderings is 30.5


In [9]:
## Plotting round 2
## Plot things for global reorder

gr()

colors_orig = [RGB(0.1,0.1,1) RGB(0.1,1,0.1) RGB(1,0.1,0.1)]

p2a = plot(1:(nNodes+1),bettiCurve_orig_mean, c=colors_orig, label = ["B_1" "B_2" "B_3"], linewidth = 2, ribbon = bettiCurveStd_orig)
plot!(1:(nNodes+1),bettiCurve_r_mean,xlabel = "nNodes", ylabel = "Betti_k",title = graph_name,
    c = colors_r, linewidth = 2, ribbon = bettiCurveStd_r, label = ["B_1 reord" "B_2 reord" "B_3 reord"],ylim = (0,yLim), framestyle =:box)
    
p2b = boxplot([1],dmat_sampled[:,1], c = colors_orig[1], markersize = 1)
boxplot!([2],dmat_glob_r[:,1], c = colors_1[1], markersize = 1)
boxplot!([3],dmat_glob_sampledR[:,1], c = colors_2[1], markersize = 1)
boxplot!([4],dmat_sampled[:,2], c = colors_orig[2], markersize = 1)
boxplot!([5],dmat_glob_r[:,2], c = colors_1[2], markersize = 1)
boxplot!([6],dmat_glob_sampledR[:,2], c = colors_2[2], markersize = 1)
boxplot!([7],dmat_sampled[:,3], c = colors_orig[3], markersize = 1)
boxplot!([8],dmat_glob_r[:,3], c =colors_1[3], markersize = 1)
boxplot!([9],dmat_glob_sampledR[:,3],xlim = (0,10), ylabel = "Distance", c = colors_2[3], markersize = 1, framestyle =:box)




p2c = boxplot([1],bettiCurveDistances[:,1], c = colors_orig[1], markersize = 1)
boxplot!([2],bettiCurveDistances_glob[:,1], c = colors_1[1], markersize = 1)
boxplot!([3],bettiCurveDistances_globSampled[:,1], c = colors_2[1], markersize = 1)
boxplot!([4],bettiCurveDistances[:,2], c = colors_orig[2], markersize = 1)
boxplot!([5],bettiCurveDistances_glob[:,2], c = colors_1[2], markersize = 1)
boxplot!([6],bettiCurveDistances_globSampled[:,2],c = colors_2[2], markersize = 1)
boxplot!([7],bettiCurveDistances[:,3], c = colors_orig[3], markersize = 1)
boxplot!([8],bettiCurveDistances_glob[:,3], c = colors_1[3], markersize = 1)
boxplot!([9],bettiCurveDistances_globSampled[:,3], ylabel = "Betti Diffs", xlim = (0,10), c = colors_2[3], markersize = 1, framestyle =:box)

p2all = plot(p2a,p2b,p2c,layout=(3,1),margin = 10mm,size= (400,900), legend=false)

#display(p2all)
savefig("$(graph_name)_GlobReorder.pdf")      

## Possibly have to load StatPlots, too?

GC.gc()


UnequalVarianceTTest(dmat_glob_sampledR[:,1], dmat_glob[:,1])


Two sample t-test (unequal variance)
------------------------------------
Population details:
    parameter of interest:   Mean difference
    value under h_0:         0
    point estimate:          -0.4216816831683179
    95% confidence interval: (-0.4869, -0.3564)

Test summary:
    outcome with 95% confidence: reject h_0
    two-sided p-value:           <1e-35

Details:
    number of observations:   [10000,10100]
    t-statistic:              -12.670428545278185
    degrees of freedom:       19682.618891376394
    empirical standard error: 0.03328077512622599
